# Neo4j MCP Agent with Strands Agents

Query a Neo4j graph database using natural language with **AWS Strands Agents** and **AgentCore Gateway MCP**.

## Overview

This notebook demonstrates how to build an AI agent that can query a Neo4j graph database using natural language. The agent uses:

- **Strands Agents**: AWS's lightweight agent SDK for building AI agents
- **Model Context Protocol (MCP)**: A standard protocol for connecting AI agents to external tools
- **Neo4j MCP Server**: Exposes Neo4j databases through MCP, enabling schema introspection and Cypher query execution
- **Amazon Bedrock**: Provides the Claude LLM for reasoning and query generation

**How it works**: When you ask a question, the agent first retrieves the database schema to understand what data exists, then generates and executes Cypher queries to answer your question.

## Prerequisites

Before running this notebook, ensure you have:

1. **Configured `CONFIG.txt`** with the following values:
   - `MCP_GATEWAY_URL`: The AgentCore Gateway endpoint URL
   - `MCP_ACCESS_TOKEN`: Your authentication token for the gateway
   
2. **Obtained credentials** either from:
   - Your workshop host, or
   - Deploying the Neo4j MCP Server yourself using [aws-starter](https://github.com/neo4j-partners/aws-starter)

3. **AWS credentials** configured for Amazon Bedrock access (handled automatically in SageMaker)

See the [README](./README.md) for detailed setup instructions.

## 1. Setup

Install the required packages for Strands Agents and MCP connectivity. 

- **strands-agents**: The core agent framework
- **strands-agents-tools**: Pre-built tool integrations
- **mcp**: The Model Context Protocol client library
- **httpx**: HTTP client used by the MCP transport

In [ ]:
%pip install strands-agents strands-agents-tools mcp httpx -q

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp.mcp_client import MCPClient
from mcp.client.streamable_http import streamablehttp_client

print("Imports OK")

## 2. Configuration

Load the model and MCP Gateway credentials from `CONFIG.txt`. The configuration includes:

- **MODEL_ID**: The Bedrock model to use for reasoning (e.g., Claude)
- **REGION**: AWS region for Bedrock API calls
- **MCP_GATEWAY_URL**: The AgentCore Gateway endpoint that routes requests to the Neo4j MCP Server
- **MCP_ACCESS_TOKEN**: Bearer token for authenticating with the gateway

The gateway URL and token are provided by your workshop host or generated when you deploy your own MCP server.

In [ ]:
# Load configuration from CONFIG.txt
from dotenv import load_dotenv
import os

load_dotenv("../CONFIG.txt")

MODEL_ID = os.getenv("MODEL_ID")
REGION = os.getenv("REGION", "us-west-2")
GATEWAY_URL = os.getenv("MCP_GATEWAY_URL")
ACCESS_TOKEN = os.getenv("MCP_ACCESS_TOKEN")

# Derive BASE_MODEL_ID for Claude inference profiles
# us.anthropic.claude-* -> anthropic.claude-*
if MODEL_ID and MODEL_ID.startswith("us.anthropic."):
    BASE_MODEL_ID = MODEL_ID.replace("us.anthropic.", "anthropic.")
else:
    BASE_MODEL_ID = None

print(f"Model:   {MODEL_ID}")
if BASE_MODEL_ID:
    print(f"Base:    {BASE_MODEL_ID}")
print(f"Region:  {REGION}")

# Validate gateway credentials
if not GATEWAY_URL or "your-" in GATEWAY_URL:
    print("\nWARNING: Set MCP_GATEWAY_URL in CONFIG.txt before running MCP cells")
elif not ACCESS_TOKEN or "your-" in ACCESS_TOKEN:
    print("\nWARNING: Set MCP_ACCESS_TOKEN in CONFIG.txt before running MCP cells")
else:
    print(f"Gateway: {GATEWAY_URL[:50]}...")
    print("Configuration OK!")

## 3. Initialize Model & MCP Client

Set up the Bedrock model and MCP client connection.

**Key concepts:**

- **BedrockModel**: Wrapper that handles API calls to Amazon Bedrock for LLM inference
- **MCPClient**: Manages the connection to the MCP server and provides tool discovery/invocation
- **Transport Factory**: A function that creates fresh HTTP connections with authentication headers. The factory pattern ensures each request gets a properly authenticated connection.

The `streamablehttp_client` is MCP's modern HTTP transport—it replaces the older SSE-based approach and works better with load balancers and proxies.

In [ ]:
# Bedrock model
model = BedrockModel(
    model_id=MODEL_ID,
    region_name=REGION,
    temperature=0,
)


# Token getter (called each time transport is created)
def get_token():
    return ACCESS_TOKEN


# Transport factory - returns fresh streamablehttp_client each call
def create_streamable_http_transport():
    return streamablehttp_client(
        GATEWAY_URL,
        headers={"Authorization": f"Bearer {get_token()}"}
    )


# MCP client with transport factory
mcp_client = MCPClient(create_streamable_http_transport)
print("MCP client ready")

## 4. Test MCP Connection

Verify connectivity to the MCP server by listing available tools. This confirms:

1. The gateway URL is correct and reachable
2. The access token is valid
3. The Neo4j MCP Server is running and responding

You should see two tools: `get-schema` (for retrieving database structure) and `read-cypher` (for executing queries).

In [ ]:
with mcp_client:
    tools = mcp_client.list_tools_sync()
    print(f"Connected! Found {len(tools)} tools:")
    for tool in tools:
        print(f"  - {tool.tool_spec['name']}")

## 5. Create Agent & Query Function

Create the agent and a helper function to query the database.

**How the agent works:**

1. The `query()` function opens an MCP connection and discovers available tools
2. It creates a Strands `Agent` with the Bedrock model, MCP tools, and a system prompt
3. When you ask a question, the agent:
   - Analyzes your question using the LLM
   - Decides which tools to call (typically `get-schema` first, then `read-cypher`)
   - Executes the tools via MCP
   - Synthesizes the results into a natural language response

**System prompt**: Instructs the agent to always get the schema first before querying. This is crucial because the LLM needs to know what node labels and relationship types exist to generate valid Cypher.

In [ ]:
SYSTEM_PROMPT = """You are a Neo4j database assistant. You can:
- Get the database schema
- Run read-only Cypher queries

Always get the schema first, then query based on actual labels/relationships.
Be concise. Format results clearly."""


def query(question: str) -> str:
    """Query the Neo4j database via MCP."""
    print(f"Q: {question}")
    print("-" * 60)
    
    with mcp_client:
        tools = mcp_client.list_tools_sync()
        agent = Agent(
            model=model,
            tools=tools,
            system_prompt=SYSTEM_PROMPT,
        )
        result = agent(question)
    
    print(f"\nA: {result}")
    return str(result)

## 6. Demo Queries

Run these sample queries to see the agent in action. Watch the output to observe:

- **Tool calls**: Which MCP tools the agent invokes and in what order
- **Cypher generation**: The queries the LLM creates based on the schema
- **Result synthesis**: How raw data is transformed into natural language answers

Each query demonstrates a different capability—from simple schema inspection to relationship traversal.

In [ ]:
_ = query("What is the database schema?")

In [ ]:
_ = query("How many nodes are there by label?")

In [ ]:
_ = query("Show 5 sample records from the most populated node type.")

## 7. Your Query

Try your own natural language questions! Some ideas based on the manufacturing dataset:

- "What requirements does the HVB_3900 component have?"
- "What defects have been detected and what are their severities?"
- "Which technology domains does the R2D2 product cover?"
- "What components belong to the Electric Powertrain domain?"
- "What changes affect battery-related requirements?"

The agent will figure out the right Cypher query—you don't need to know the syntax.

In [ ]:
# _ = query("Your question here")